In [1]:
win_size = 9

In [2]:
import sys
sys.path.append('../')
from time import sleep
from models.attention_series import Grade_regressor
import spectral
from configs.training_cfg import device
import torch
import ast
import numpy as np
from PIL import Image
import pandas as pd
from spectral import imshow
from matplotlib import rcParams
import seaborn as sns
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

spectral.settings.envi_support_nonlowercase_params = True

rcParams['figure.figsize'] = 11,19

pre_list = []
erro_list = []

In [3]:
# 测试样本绝对误差
for k in range(1):
    model = Grade_regressor().to(device).eval()
    model.load_state_dict(torch.load("..\\ckpt\\RELU+ALLINDEX_TANH_LINEAR_50000.pt".format(k+1)))
    result = []
    err = 0
    err_count = 0 
    pool = torch.nn.AvgPool2d(win_size,win_size)
    mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88],[184,61,186]]

    spec_id = [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64]    # 选择评估的成像光谱图片编号


    for id in spec_id:
        img = spectral.envi.open("E:\\d盘备份\\可见光部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr".format(id))

        # 根据模型使用波段选择
        # img_data = torch.Tensor(img.asarray()/6000)[:,:,:-4]
        img_data = torch.Tensor(img.asarray()/6000)[:,:,:]
        mask = np.array(Image.open("E:\\d盘备份\\可见光部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr_mask.png".format(id)))
        if mask.shape[2] == 4:
            mask = mask[:,:,:-1]
        gt_TFe = ast.literal_eval(img.metadata['gt_TFe'])
        gt = torch.Tensor(gt_TFe)
        gts = gt.__len__()

        with torch.no_grad():
            img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
            row,col,_ = img_data.shape
            heat_map = []

            for i in range(row):
                heat_map.append(model(img_data[i].to(device)).squeeze(1).unsqueeze(0).to("cpu"))   # 只评估Tfe
                torch.cuda.empty_cache()

        heat_map = torch.cat(heat_map, dim=0)

        # # 可视化
        # plt.figure()#防止在上一张图上重新绘制
        # hm = sns.heatmap(heat_map.numpy(), cmap="viridis", xticklabels=False, yticklabels=False)
        # hm.figure.savefig("新可见光relu/{}号预测结果可视化.jpg".format(id))

        predict_sum = torch.zeros(gts)
        pixel_count = torch.ones(gts)/1e5

        for r in range(row):
            for c in range(col):
                for i in range(gts):
                    if mask[r*win_size+win_size//2,c*win_size+win_size//2].tolist() == mask_rgb_values[i]:
                        predict_sum[i] += heat_map[r,c] 
                        pixel_count[i] += 1

        prediction = predict_sum / pixel_count * 100

        err_list = ((prediction-gt)).tolist()
        pred_list = (prediction).tolist()
        for i in range(err_list.__len__()):
            erro_list.append(round(err_list[i],2))
            pre_list.append(round(pred_list[i], 2))
            

    # avg_err = err/(err_count)
    # print("{}k  avgmse:{}".format(k+1, avg_err))

FileNotFoundError: Unable to locate file "E:\d盘备份\可见光部分\spectral_data\64-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.

In [4]:
erro_list

[-4.1,
 8.51,
 -4.01,
 1.11,
 3.2,
 -6.35,
 -0.64,
 -0.91,
 8.29,
 -2.68,
 4.21,
 11.74,
 -0.46,
 0.0,
 1.79,
 -0.16,
 0.49,
 -1.48,
 -2.33,
 -5.43,
 1.33,
 -0.13,
 -3.68,
 1.47,
 -8.33,
 -4.44,
 0.48,
 3.88,
 1.85,
 4.24,
 -3.4,
 -3.43,
 -2.93,
 -0.13,
 -1.83,
 4.42,
 -7.64,
 2.91,
 -8.9,
 2.9,
 -2.72,
 -6.85,
 6.75,
 -4.95,
 -6.44,
 1.48,
 6.09,
 0.85,
 -1.43,
 2.89,
 -0.41,
 -5.0,
 5.45,
 6.54,
 4.83,
 5.59,
 -8.11,
 -1.25,
 2.3,
 1.7,
 -5.24,
 -2.9,
 -4.43,
 0.12,
 1.32,
 -5.57,
 0.19,
 4.33,
 -4.47,
 0.82,
 -3.84,
 -6.3,
 0.29,
 -4.61,
 5.83,
 2.86,
 2.83,
 6.82,
 -5.0,
 0.41,
 -2.59,
 2.81,
 -5.57,
 2.64,
 8.1,
 -0.78,
 -7.37,
 -1.23,
 -1.92,
 0.95,
 -4.06,
 5.02,
 -0.46,
 -0.12,
 -4.56,
 -3.51,
 -0.36,
 -5.34,
 -6.28,
 -4.31,
 -7.04,
 2.96,
 0.6,
 -3.36,
 7.49,
 -4.83,
 -0.96,
 6.27,
 -6.23,
 -1.28,
 6.69,
 -3.32,
 1.76,
 4.65,
 5.16,
 8.63,
 -0.22,
 -2.86,
 2.91,
 -0.91,
 -2.71,
 1.99,
 -2.38,
 2.85,
 5.78,
 -1.06,
 3.94,
 -4.38,
 2.76,
 -4.48,
 3.43,
 -1.35,
 0.03,
 -1.5,
 10.

In [5]:
pre_list

[24.97,
 35.63,
 27.24,
 27.46,
 36.58,
 25.24,
 30.47,
 21.25,
 45.64,
 30.63,
 25.84,
 44.43,
 23.55,
 16.21,
 32.95,
 23.99,
 13.94,
 29.15,
 29.22,
 25.73,
 12.73,
 29.36,
 26.95,
 13.31,
 29.19,
 23.09,
 20.16,
 38.47,
 26.52,
 21.19,
 23.45,
 28.13,
 16.4,
 28.55,
 29.33,
 22.43,
 58.06,
 16.36,
 56.8,
 16.35,
 29.94,
 24.31,
 30.76,
 30.36,
 24.19,
 25.63,
 21.44,
 27.7,
 28.52,
 22.03,
 28.27,
 29.72,
 21.89,
 25.34,
 26.2,
 27.06,
 26.02,
 33.12,
 22.8,
 26.01,
 34.74,
 34.87,
 24.64,
 35.47,
 23.45,
 29.69,
 23.3,
 28.41,
 28.78,
 20.5,
 26.95,
 24.61,
 22.44,
 24.6,
 27.04,
 38.93,
 33.27,
 38.24,
 28.27,
 36.8,
 36.0,
 40.45,
 26.17,
 27.36,
 28.65,
 66.05,
 30.25,
 63.33,
 27.98,
 28.48,
 28.06,
 42.66,
 36.64,
 36.98,
 30.72,
 31.08,
 37.67,
 61.25,
 59.58,
 28.46,
 28.89,
 22.44,
 24.81,
 27.08,
 26.29,
 27.3,
 30.15,
 23.22,
 22.06,
 27.6,
 23.13,
 27.93,
 28.88,
 28.26,
 26.79,
 41.32,
 28.66,
 30.45,
 27.12,
 23.83,
 30.8,
 28.59,
 28.97,
 26.97,
 27.94,
 27.82,
 27.5